MODEL	
Author/Vintage/Updates	
Type of Model	
Time Period/Temporal Resolution	
Geographic Coverage/Spatial Resolution	
Availability (Subscription vs Consultant)	
Cost	
Energy sector coverage	
Economic outputs	
Policy representation (state and federal vs state regulatory, IRP, RTO level)	
RE representation	
Storage representation	
Demand-side representation (DR, vehicle charging, DG)	
Transmission representation	
user_friendly	
Data availability	
Results presentation	
Who uses what	
Training/consulting and documentation/manual availabiltiy

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)

## Clean the Dataset

* Replace NaNs
* Strip erroneous symbols (e.g., `+`)

In [2]:
model_df = pd.read_csv('results/esom_database_raw.csv', encoding='utf-8', index_col='Model')

In [3]:
model_df.fillna(str(-999), inplace=True)

In [4]:
print(f"The parser found {len(model_df.index)} distinct energy models on the openmod wiki.")

The parser found 90 distinct energy models on the openmod wiki.


In [5]:
model_df.columns

Index(['institution', 'doi', 'citation', 'computation time comments',
       'computation time minutes', 'contact email', 'contact persons',
       'data availability', 'decisions', 'deterministic', 'full model name',
       'georegions', 'georesolution', 'is suited for many scenarios',
       'license', 'math model type', 'math model type shortdesc',
       'model class', 'model source public', 'modelling software',
       'open future', 'open source licensed', 'processing software',
       'report references', 'sectors', 'source download', 'technologies',
       'text description', 'time resolution', 'website', 'objective', 'url',
       'authors', 'example research questions', 'math objective',
       'network coverage', 'number of variables'],
      dtype='object')

In [6]:
cols_of_interest = ['data availability', 'decisions', 'deterministic',
                    'georesolution', 'is suited for many scenarios',
                    'math model type',
                    'model source public', 'modelling software',
                    'open source licensed', 'processing software',
                    'sectors','technologies','objective','network coverage']

In [7]:
model_df = model_df[cols_of_interest]

In [8]:
strip_str = lambda x: x.strip(' +')
strip_series = lambda x: x.apply(strip_str)

In [9]:
model_df = model_df.apply(strip_series)

In [10]:
model_df[['modelling software']].head()

,modelling software
Model,
AMIRIS,Java
ASAM,Python (Pyomo)
Antares-Simulator,"C++, C"
AnyMOD,Julia/JuMP
Backbone,GAMS


## Process Data

* Filter for the frameworks that have open licenses AND have open source code.

In [11]:
open_mask = ((model_df['open source licensed']=='true') & (model_df['model source public']=='true'))

In [12]:
model_df = model_df[open_mask]

* Filter for frameworks that are suitable for many scenarios

In [13]:
model_df['is suited for many scenarios'].unique()

array(['true', 'false'], dtype=object)

In [14]:
scenario_mask = (model_df['is suited for many scenarios'] == 'true')

In [15]:
model_df = model_df[scenario_mask]

* Filter for the frameworks that use Python as a modelling software.

Python is a comprehensible high-level programming language and its popularity makes it flexible and useful for a variety of tasks.

In [16]:
python_mask = ((model_df['modelling software'].str.contains('python', case=False)) | 
               (model_df['processing software'].str.contains('python', case=False)) |
               (model_df['processing software'].str.contains('pandas', case=False)) |
               (model_df['processing software'].str.contains('sqlite', case=False)))

In [17]:
model_df[python_mask][['objective','modelling software', 'processing software']]

,objective,modelling software,processing software
Model,,,
AMIRIS,-999,Java,Python
ASAM,-999,Python (Pyomo),"Python, PyPSA, Mesa"
Antares-Simulator,"socio-economic welfare, investment costs, greenhouse gas emissions","C++, C","Python, TypeScript"
CAPOW,Cost minimization,Python (Pyomo),-999
Calliope,"User-dependent, including financial cost, CO2, and water consumption",Python (Pyomo),Python (pandas et al)
Demod,Assess domestic energy demand evolution and demand-side-management scenarios,Python,Python
Dispa-SET,Minimization of operational costs,"Python (Pyomo), GAMS",Python
Ficus,costs,Python (Pyomo),Python (pandas et al)
GridCal,Match generation to demand and minimise cost,Python,Python


In [18]:
model_df = model_df[python_mask]

In [19]:
model_df

,data availability,decisions,deterministic,georesolution,is suited for many scenarios,math model type,model source public,modelling software,open source licensed,processing software,sectors,technologies,objective,network coverage
Model,,,,,,,,,,,,,,
AMIRIS,all,dispatch,"stochastic, perfect foresight, deterministic",National,true,Simulation + and Agent-based,true,Java,true,Python,electricity,Renewables + and Conventional Generation,-999,-999
ASAM,some,dispatch,-999,Individual power stations,true,Simulation + and Agent-based,true,Python (Pyomo),true,"Python, PyPSA, Mesa","Electricity +, Electricity Market + and Electric power",-999,-999,-999
Antares-Simulator,some,dispatch + and investment,"Monte-Carlo methods, myopic week-ahead foresight",NUTS0 - NUTS2,true,Optimization + and Simulation,true,"C++, C",true,"Python, TypeScript","Electricity +, Methane +, Hydrogen + and Heat","Renewables +, Conventional Generation + and CHP","socio-economic welfare, investment costs, greenhouse gas emissions","transmission +, DC load flow + and net transfer capacities"
CAPOW,all,dispatch,Short-term and long-term stochastics are available,Zonal,true,Simulation,true,Python (Pyomo),true,-999,Electric power,Renewables + and Conventional Generation,Cost minimization,transmission
Calliope,some,dispatch + and investment,Deterministic; stochastic programming add-on,User-dependent,true,Optimization,true,Python (Pyomo),true,Python (pandas et al),User-dependent,"Renewables +, Conventional Generation + and CHP","User-dependent, including financial cost, CO2, and water consumption","transmission +, distribution + and net transfer capacities"
Demod,some,-999,Not yet implemented,depending on input data,true,Simulation,true,Python,true,Python,end-use demand,-999,Assess domestic energy demand evolution and demand-side-management scenarios,-999
Dispa-SET,all,dispatch,Through proper sizing of reserve needs,NUTS1,true,Optimization,true,"Python (Pyomo), GAMS",true,Python,Power system,"Renewables +, Conventional Generation + and CHP",Minimization of operational costs,net transfer capacities
Ficus,some,dispatch + and investment,None,-999,true,Optimization,true,Python (Pyomo),true,Python (pandas et al),"electricity +, heating + and ...","Renewables +, Conventional Generation + and CHP",costs,-999
GridCal,all,-999,"Deterministic, stochastic",-999,true,Optimization + and Simulation,true,Python,true,Python,Electricity,Conventional Generation,Match generation to demand and minimise cost,"transmission +, distribution +, AC load flow + and DC load flow"
